<a href="https://colab.research.google.com/github/Annie-Yeeun-Jang/dacon_timeseries/blob/main/final_model/dacon_timeseries_private.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Private 평가용 모델 학습


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
from google.colab import drive
from datetime import datetime, timedelta

import math
import time
import random

import lightgbm as lgb
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.externals import joblib

drive.mount('./gdrive', force_remount=True)
plt.style.use("fivethirtyeight")
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Mounted at ./gdrive


In [56]:
dangjin_fcst = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/new_dangjin_fcst.csv")
ulsan_fcst = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/new_ulsan_fcst.csv")
energy = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/new_energy.csv")

In [57]:
dangjin_fcst

,Forecast_time,Temperature,PrecipitationForm,PrecipitationProb,Humidity,WindSpeed,WindDirection,Cloud,Precipitation,Snow,방위각,고도,적경,적위
0,2016-12-02 00:00:00,0.4,0.0,0.0,67.0,8.6,356.0,1.0,NaN,NaN,340 03 04.02,-74 04 10.40,16 32 56.04,-21 55 49.38
1,2016-12-02 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,030 19 39.37,-72 55 00.56,16 33 06.98,-21 56 11.79
2,2016-12-02 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,060 54 26.99,-64 12 35.47,16 33 17.92,-21 56 34.29
3,2016-12-02 03:00:00,-0.7,0.0,0.0,76.0,2.7,341.0,1.0,NaN,NaN,077 02 49.13,-53 01 39.40,16 33 28.84,-21 56 56.88
4,2016-12-02 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,087 49 23.81,-41 10 38.68,16 33 39.74,-21 57 19.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37196,2021-02-28 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,274 36 31.31,-19 28 58.57,22 43 34.22,-8 04 37.23
37197,2021-02-28 21:00:00,7.0,0.0,30.0,70.0,1.3,18.0,4.0,NaN,NaN,284 38 48.32,-31 16 00.31,22 43 43.68,-8 03 40.40
37198,2021-02-28 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,296 55 39.10,-42 26 50.80,22 43 53.17,-8 02 43.58
37199,2021-02-28 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,313 32 08.08,-52 14 15.85,22 44 02.67,-8 01 46.80


In [3]:
#여기서 불러와서 concat하기
#chch_sun=pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/prep_chch_sun.csv")
#ulsan_sun=pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/prep_ulsan_sun.csv")
#chch_sun.rename(columns={'일시':'Forecast_time','일조(hr)':'일조','일사(MJ/m2)':'일사'}, inplace=True)
#ulsan_sun.rename(columns={'일시':'Forecast_time','일조(hr)':'일조','일사(MJ/m2)':'일사'}, inplace=True)

In [4]:
energy['time'] = pd.to_datetime(dangjin_fcst['Forecast_time'][:len(energy)])
energy['year'] = energy['time'].dt.year
energy['month'] = energy['time'].dt.month
energy['day'] = energy['time'].dt.day
energy['hour'] = energy['time'].dt.hour
energy['year-month-day'] = energy['year'].astype(str) + "-" + energy['month'].astype(str) + "-" + energy['day'].astype(str)

In [5]:
date = dangjin_fcst['Forecast_time']

In [6]:
dangjin_fcst['Temperature'] = dangjin_fcst['Temperature'].interpolate()  #interpolate() 구간 따로 안정하고 column 단위로 통째로 적용하면 되는거 맞음
dangjin_fcst['PrecipitationForm'] = dangjin_fcst['PrecipitationForm'].interpolate()
dangjin_fcst['PrecipitationProb'] = dangjin_fcst['PrecipitationProb'].interpolate()
dangjin_fcst['Humidity'] = dangjin_fcst['Humidity'].interpolate()
dangjin_fcst['WindSpeed'] = dangjin_fcst['WindSpeed'].interpolate()
dangjin_fcst['Cloud'] = dangjin_fcst['Cloud'].interpolate()
dangjin_fcst['Precipitation'] = dangjin_fcst['Precipitation'].interpolate()
dangjin_fcst['Snow'] = dangjin_fcst['Snow'].interpolate()
#dangjin_fcst['WindDirection'] = dangjin_fcst['WindDirection'].interpolate() => 방향으로 바꾸는 작업 해주려고

ulsan_fcst['Temperature'] = ulsan_fcst['Temperature'].interpolate()
ulsan_fcst['PrecipitationForm'] = ulsan_fcst['PrecipitationForm'].interpolate()
ulsan_fcst['PrecipitationProb'] = ulsan_fcst['PrecipitationProb'].interpolate()
ulsan_fcst['Humidity'] = ulsan_fcst['Humidity'].interpolate()
ulsan_fcst['WindSpeed'] = ulsan_fcst['WindSpeed'].interpolate()
ulsan_fcst['Cloud'] = ulsan_fcst['Cloud'].interpolate()
ulsan_fcst['Precipitation'] = ulsan_fcst['Precipitation'].interpolate()
ulsan_fcst['Snow'] = ulsan_fcst['Snow'].interpolate()
#ulsan_fcst['WindDirection'] = ulsan_fcst['WindDirection'].interpolate() 선형보간 안해서 nan 많음

In [7]:
#Wind Direction 항목 angle에서 방향표시로 바꿔주기
def angle_to_dir(x):
    if pd.isnull(x):
        return np.nan
    elif x >= 22.5 and x <67.5:
        return 'NE'
    elif x >= 67.5 and x < 112.5:
        return 'E'
    elif x >= 112.5 and x < 157.5:
        return 'SE'
    elif x >= 157.5 and x < 202.5:
        return 'S'
    elif x >= 202.5 and x < 247.5:
        return 'WS'
    elif x >= 247.5 and x < 292.5:
        return 'W'
    elif x >= 292.5 and x < 337.5:
        return 'NW'
    elif x >= 337.5 or x < 22.5:
        return 'N'

def angle_to_cos(x):
    return np.cos(np.pi/180*(x-90))
def angle_to_sin(x):
    return np.sin(np.pi/180*(x+90))

dangjin_fcst['WindDirection_x'] = dangjin_fcst['WindDirection'].apply(angle_to_cos)  #Q. 어차피 다 나눠놨는데싸인 코싸인은 왜 씌우는것..?
dangjin_fcst['WindDirection_y'] = dangjin_fcst['WindDirection'].apply(angle_to_sin)
dangjin_fcst['Wind_x'] = dangjin_fcst['WindDirection_x'] * dangjin_fcst['WindSpeed']
dangjin_fcst['Wind_y'] = dangjin_fcst['WindDirection_y'] * dangjin_fcst['WindSpeed']

ulsan_fcst['WindDirection_x'] = ulsan_fcst['WindDirection'].apply(angle_to_cos)
ulsan_fcst['WindDirection_y'] = ulsan_fcst['WindDirection'].apply(angle_to_sin)
ulsan_fcst['Wind_x'] = ulsan_fcst['WindDirection_x'] * ulsan_fcst['WindSpeed']
ulsan_fcst['Wind_y'] = ulsan_fcst['WindDirection_y'] * ulsan_fcst['WindSpeed']


dangjin_fcst['WindDirection'] = dangjin_fcst['WindDirection'].interpolate()
ulsan_fcst['WindDirection'] = ulsan_fcst['WindDirection'].interpolate()

dangjin_fcst['WindDirection_catg'] = dangjin_fcst['WindDirection'].apply(angle_to_dir)
ulsan_fcst['WindDirection_catg'] = ulsan_fcst['WindDirection'].apply(angle_to_dir)

In [8]:
dangjin_fcst['Wind_x'] = dangjin_fcst['Wind_x'].interpolate()
dangjin_fcst['Wind_y'] = dangjin_fcst['Wind_y'].interpolate()

ulsan_fcst['Wind_x'] = ulsan_fcst['Wind_x'].interpolate()
ulsan_fcst['Wind_y'] = ulsan_fcst['Wind_y'].interpolate()

In [9]:
#dangjin_fcst=pd.merge(dangjin_fcst, chch_sun, how='left', on='Forecast_time')
#ulsan_fcst=pd.merge(ulsan_fcst, ulsan_sun, how='left', on='Forecast_time')

In [10]:
dangjin_fcst['hour'] = pd.to_datetime(dangjin_fcst['Forecast_time']).dt.hour
dangjin_fcst['day'] = pd.to_datetime(dangjin_fcst['Forecast_time']).dt.day
dangjin_fcst['month'] = pd.to_datetime(dangjin_fcst['Forecast_time']).dt.month

ulsan_fcst['hour'] = pd.to_datetime(ulsan_fcst['Forecast_time']).dt.hour
ulsan_fcst['day'] = pd.to_datetime(ulsan_fcst['Forecast_time']).dt.day
ulsan_fcst['month'] = pd.to_datetime(ulsan_fcst['Forecast_time']).dt.month

In [11]:
#이슬점 직접 계산해서 추가
b = 17.62
c = 243.12
gamma = (b * dangjin_fcst['Temperature'] /(c + dangjin_fcst['Temperature'])) + np.log(dangjin_fcst['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
dangjin_fcst['DewPoint'] = dewpoint  

gamma = (b * ulsan_fcst['Temperature'] /(c + ulsan_fcst['Temperature'])) + np.log(ulsan_fcst['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
ulsan_fcst['DewPoint'] = dewpoint

In [12]:
#형태 다듬기 (?)
dangjin_fcst['방위각'] = dangjin_fcst['방위각'].apply(lambda x : int(x[:3]))
dangjin_fcst['고도'] = dangjin_fcst['고도'].apply(lambda x : int(x.split(" ")[0]))
dangjin_fcst['적경'] = dangjin_fcst['적경'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
dangjin_fcst['적위'] = dangjin_fcst['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

ulsan_fcst['방위각'] = ulsan_fcst['방위각'].apply(lambda x : int(x[:3]))
ulsan_fcst['고도'] = ulsan_fcst['고도'].apply(lambda x : int(x.split(" ")[0]))
ulsan_fcst['적경'] = ulsan_fcst['적경'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
ulsan_fcst['적위'] = ulsan_fcst['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

In [13]:
using_col = ['Temperature', 'PrecipitationForm', 'PrecipitationProb', 'DewPoint',
             'Humidity', 'Cloud', 'Precipitation', 'Snow', 'Wind_x', 'Wind_y',
             'hour', 'month', '방위각', '고도', '적경', '적위']

dangjin_fcst = dangjin_fcst[using_col].fillna(0)
ulsan_fcst = ulsan_fcst[using_col].fillna(0)

In [14]:
# Shift => period만큼 데이터 옮겨서 concat
# 해당 시간대에 대해 당일 예보, 하루 전 예보, 이틀 전 예보 이렇게 column을 늘려줌
def shift(df, period):
  target = pd.DataFrame()
  for i in range(period, -1, -1): #period부터 거꾸로 1칸씩 (71부터 0까지 출력)
    tmp = df.shift(i) #i는 period
    target = pd.concat([target, tmp], axis = 1)
  return target

In [15]:
dangjin_fcst

,Temperature,PrecipitationForm,PrecipitationProb,DewPoint,Humidity,Cloud,Precipitation,Snow,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위
0,0.400000,0.0,0.0,-5.020566,67.000000,1.0,0.0,0.0,-0.599906,8.579051,0,12,340,-74,16.3256,-21.5549
1,0.033333,0.0,0.0,-4.791717,70.000000,1.0,0.0,0.0,-0.692948,6.570334,1,12,30,-72,16.3306,-21.5611
2,-0.333333,0.0,0.0,-4.587913,73.000000,1.0,0.0,0.0,-0.785991,4.561617,2,12,60,-64,16.3317,-21.5634
3,-0.700000,0.0,0.0,-4.407317,76.000000,1.0,0.0,0.0,-0.879034,2.552900,3,12,77,-53,16.3328,-21.5656
4,-0.933333,0.0,0.0,-3.950599,80.000000,1.0,0.0,0.0,-0.761132,2.240865,4,12,87,-41,16.3339,-21.5719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37196,7.666667,0.0,30.0,1.851700,66.666667,4.0,0.0,0.0,0.163620,1.069717,20,2,274,-19,22.4334,-8.0437
37197,7.000000,0.0,30.0,1.897402,70.000000,4.0,0.0,0.0,0.401722,1.236373,21,2,284,-31,22.4343,-8.0340
37198,6.666667,0.0,30.0,1.241159,68.333333,4.0,0.0,0.0,0.574228,1.130662,22,2,296,-42,22.4353,-8.0243
37199,6.333333,0.0,30.0,0.579324,66.666667,4.0,0.0,0.0,0.746733,1.024950,23,2,313,-52,22.4402,-8.0146


In [16]:
dangjin_fcst = shift(dangjin_fcst, 71)  
ulsan_fcst = shift(ulsan_fcst, 71)      
                                        

In [17]:
dangjin_fcst

,Temperature,PrecipitationForm,PrecipitationProb,DewPoint,Humidity,Cloud,Precipitation,Snow,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,PrecipitationForm,PrecipitationProb,DewPoint,Humidity,Cloud,Precipitation,Snow,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,PrecipitationForm,PrecipitationProb,DewPoint,Humidity,Cloud,Precipitation,Snow,...,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,PrecipitationForm,PrecipitationProb,DewPoint,Humidity,Cloud,Precipitation,Snow,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,PrecipitationForm,PrecipitationProb,DewPoint,Humidity,Cloud,Precipitation,Snow,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,0.0,0.0,-5.020566,67.000000,1.0,0.0,0.0,-0.599906,8.579051,0,12,340,-74,16.3256,-21.5549
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,0.0,0.0,-5.020566,67.000000,1.0,0.0,0.0,-0.599906,8.579051,0.0,12.0,340.0,-74.0,16.3256,-21.5549,0.033333,0.0,0.0,-4.791717,70.000000,1.0,0.0,0.0,-0.692948,6.570334,1,12,30,-72,16.3306,-21.5611
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.599906,8.579051,0.0,12.0,340.0,-74.0,16.3256,-21.5549,0.033333,0.0,0.0,-4.791717,70.000000,1.0,0.0,0.0,-0.692948,6.570334,1.0,12.0,30.0,-72.0,16.3306,-21.5611,-0.333333,0.0,0.0,-4.587913,73.000000,1.0,0.0,0.0,-0.785991,4.561617,2,12,60,-64,16.3317,-21.5634
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.692948,6.570334,1.0,12.0,30.0,-72.0,16.3306,-21.5611,-0.333333,0.0,0.0,-4.587913,73.000000,1.0,0.0,0.0,-0.785991,4.561617,2.0,12.0,60.0,-64.0,16.3317,-21.5634,-0.700000,0.0,0.0,-4.407317,76.000000,1.0,0.0,0.0,-0.879034,2.552900,3,12,77,-53,16.3328,-21.5656
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.785991,4.561617,2.0,12.0,60.0,-64.0,16.3317,-21.5634,-0.700000,0.0,0.0,-4.407317,76.000000,1.0,0.0,0.0,-0.879034,2.552900,3.0,12.0,77.0,-53.0,16.3328,-21.5656,-0.933333,0.0,0.0,-3.950599,80.000000,1.0,0.0,0.0,-0.761132,2.240865,4,12,87,-41,16.3339,-21.5719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37196,3.000000,0.0,30.0,-0.115438,80.000000,4.0,0.0,0.0,0.414110,1.545481,21.0,2.0,283.0,-31.0,22.3224,-9.1106,2.333333,0.0,20.0,-1.347498,76.666667,3.0,0.0,0.0,0.684631,1.373141,22.0,2.0,295.0,-43.0,22.3233,-9.1010,1.666667,0.0,10.0,-2.596163,73.333333,2.0,0.0,0.0,...,-0.312585,0.736404,18.0,2.0,256.0,4.0,22.4315,-8.0630,8.333333,0.0,30.0,1.767587,63.333333,4.0,0.0,0.0,-0.074483,0.903060,19.0,2.0,265.0,-7.0,22.4324,-8.0534,7.666667,0.0,30.0,1.851700,66.666667,4.0,0.0,0.0,0.163620,1.069717,20,2,274,-19,22.4334,-8.0437
37197,2.333333,0.0,20.0,-1.347498,76.666667,3.0,0.0,0.0,0.684631,1.373141,22.0,2.0,295.0,-43.0,22.3233,-9.1010,1.666667,0.0,10.0,-2.596163,73.333333,2.0,0.0,0.0,0.955151,1.200801,23.0,2.0,312.0,-53.0,22.3243,-9.0914,1.000000,0.0,0.0,-3.863107,70.000000,1.0,0.0,0.0,...,-0.074483,0.903060,19.0,2.0,265.0,-7.0,22.4324,-8.0534,7.666667,0.0,30.0,1.851700,66.666667,4.0,0.0,0.0,0.163620,1.069717,20.0,2.0,274.0,-19.0,22.4334,-8.0437,7.000000,0.0,30.0,1.897402,70.0

In [18]:
dangjin_fcst_train = dangjin_fcst.iloc[:-24 * 28]
dangjin_fcst_test = dangjin_fcst.iloc[-24 * 28:]

ulsan_fcst_train = ulsan_fcst.iloc[:-24 * 28]
ulsan_fcst_test = ulsan_fcst.iloc[-24 * 28:]

In [19]:
#앞에 72개는 nan일수밖에 없음 => 해당데이터 제거
energy = energy[(dangjin_fcst_train.isnull().sum(axis = 1) <= 0).values]
ulsan_fcst_train = ulsan_fcst_train[(dangjin_fcst_train.isnull().sum(axis = 1) <= 0).values]
dangjin_fcst_train = dangjin_fcst_train[(dangjin_fcst_train.isnull().sum(axis = 1) <= 0).values]

In [20]:
#min max scaling
dangjin_min = dangjin_fcst_train.min()
dangjin_max = dangjin_fcst_train.max()

dangjin_fcst_train = (dangjin_fcst_train - dangjin_min) / (dangjin_max - dangjin_min)
dangjin_fcst_test = (dangjin_fcst_test - dangjin_min) / (dangjin_max - dangjin_min)

In [21]:
ulsan_min = ulsan_fcst_train.min()
ulsan_max = ulsan_fcst_train.max()

ulsan_fcst_train = (ulsan_fcst_train - ulsan_min) / (ulsan_max - ulsan_min)
ulsan_fcst_test = (ulsan_fcst_test - ulsan_min) / (ulsan_max - ulsan_min)

# 2D Conv 모델

In [22]:
import tensorflow as tf

In [23]:
dangjin_fcst_train.shape

(36458, 1152)

In [24]:
dangjin_fcst_train

,Temperature,PrecipitationForm,PrecipitationProb,DewPoint,Humidity,Cloud,Precipitation,Snow,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,PrecipitationForm,PrecipitationProb,DewPoint,Humidity,Cloud,Precipitation,Snow,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,PrecipitationForm,PrecipitationProb,DewPoint,Humidity,Cloud,Precipitation,Snow,...,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,PrecipitationForm,PrecipitationProb,DewPoint,Humidity,Cloud,Precipitation,Snow,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,PrecipitationForm,PrecipitationProb,DewPoint,Humidity,Cloud,Precipitation,Snow,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위
71,0.293878,0.0,0.000000,0.321815,0.633333,0.000000,0.000000,0.000000,0.372243,0.749774,0.000000,1.0,0.968571,0.006667,0.691902,0.036692,0.286395,0.0,0.000000,0.326229,0.666667,0.000000,0.000000,0.000000,0.370026,0.707605,0.043478,1.0,0.082857,0.020000,0.692114,0.036558,0.278912,0.0,0.000000,0.330161,0.700000,0.000000,0.000000,0.000000,...,0.401532,0.553359,0.913043,1.0,0.777143,0.213333,0.697272,0.022931,0.417687,0.000000,0.333333,0.508851,0.870370,1.0,0.003333,0.0,0.397723,0.556202,0.956522,1.0,0.811429,0.133333,0.697319,0.022804,0.421088,0.000000,0.333333,0.513063,0.874074,1.0,0.004167,0.0,0.393915,0.559045,1.000000,1.000000,0.862857,0.060000,0.697365,0.022763
72,0.286395,0.0,0.000000,0.326229,0.666667,0.000000,0.000000,0.000000,0.370026,0.707605,0.043478,1.0,0.082857,0.020000,0.692114,0.036558,0.278912,0.0,0.000000,0.330161,0.700000,0.000000,0.000000,0.000000,0.367809,0.665436,0.086957,1.0,0.168571,0.073333,0.692161,0.036509,0.271429,0.0,0.000000,0.333644,0.733333,0.000000,0.000000,0.000000,...,0.397723,0.556202,0.956522,1.0,0.811429,0.133333,0.697319,0.022804,0.421088,0.000000,0.333333,0.513063,0.874074,1.0,0.004167,0.0,0.393915,0.559045,1.000000,1.0,0.862857,0.060000,0.697365,0.022763,0.424490,0.000000,0.333333,0.517275,0.877778,1.0,0.005000,0.0,0.390107,0.561888,0.000000,1.000000,0.962857,0.006667,0.697412,0.022722
73,0.278912,0.0,0.000000,0.330161,0.700000,0.000000,0.000000,0.000000,0.367809,0.665436,0.086957,1.0,0.168571,0.073333,0.692161,0.036509,0.271429,0.0,0.000000,0.333644,0.733333,0.000000,0.000000,0.000000,0.365593,0.623267,0.130435,1.0,0.217143,0.146667,0.692207,0.036462,0.266667,0.0,0.000000,0.342454,0.777778,0.000000,0.000000,0.000000,...,0.393915,0.559045,1.000000,1.0,0.862857,0.060000,0.697365,0.022763,0.424490,0.000000,0.333333,0.517275,0.877778,1.0,0.005000,0.0,0.390107,0.561888,0.000000,1.0,0.962857,0.006667,0.697412,0.022722,0.416327,0.083333,0.451852,0.512755,0.888889,1.0,0.005833,0.0,0.387533,0.550541,0.043478,1.000000,0.082857,0.013333,0.697628,0.022595
74,0.271429,0.0,0.000000,0.333644,0.733333,0.000000,0.000000,0.000000,0.365593,0.623267,0.130435,1.0,0.217143,0.146667,0.692207,0.036462,0.266667,0.0,0.000000,0.342454,0.777778,0.000000,0.000000,0.000000,0.368402,0.616716,0.173913,1.0,0.245714,0.226667,0.692254,0.036326,0.261905,0.0,0.000000,0.350642,0.822222,0.000000,0.000000,0.000000,...,0.390107,0.561888,0.000000,1.0,0.962857,0.006667,0.697412,0.022722,0.416327,0.083333,0.451852,0.512755,0.888889,1.0,0.005833,0.0,0.387533,0.550541,0.043478,1.0,0.082857,0.013333,0.697628,0.022595,0.408163,0.166667,0.570370,0.508185,0.900000,1.0,0.006667,0.0,0.384958,0.539195,0.086957,1.000000,0.171429,0.073333,0.697675,0.022552
75,0.266667,0.0,0.000000,0.342454,0.777778,0.000000,0.000000,0.000000,0.368402,0.616716,0.173913,1.0,0.245714,0.226667,0.692254,0.036326,0.261905,0.0,0.000000,0.350642,0.822222,0.000000,0.000000,0.000000,0.371211,0.610166,0.217391,1.0,0.271429,0.306667,0.692301,0.036277,0.257143,0.0,0.000000,0.358262,0.866667,0.000000,0.000000,0.000000,...,0.387533,0.550541,0.043478,1.0,0.082857,0.013333,0.697628,0.022595,0.408163,0.166667,0.570370,0.508185,0.900000,1.0,0.006667,0.0,0.384958,0.539195,0.086957,1.0,0.171429,0.073333,0.697675,0.022552,0.400000,0.250000,0.688889,0.503567,0.911111,1.0,0.007500,0.0,0.382384,0.527848,0.130435,1.000000,0.217143,0.146667,0.697721

In [25]:
dangjin_fcst_train = dangjin_fcst_train.values.reshape((dangjin_fcst_train.shape[0], 72, 16, 1)) 
dangjin_fcst_test = dangjin_fcst_test.values.reshape((dangjin_fcst_test.shape[0], 72, 16, 1))

ulsan_fcst_train = ulsan_fcst_train.values.reshape((ulsan_fcst_train.shape[0], 72, 16, 1))
ulsan_fcst_test = ulsan_fcst_test.values.reshape((ulsan_fcst_test.shape[0], 72, 16, 1))

dangjin_fcst_train = np.squeeze(dangjin_fcst_train, axis = -1) 
dangjin_fcst_train = np.stack((dangjin_fcst_train,)*3, axis = -1)  

dangjin_fcst_test = np.squeeze(dangjin_fcst_test, axis = -1)
dangjin_fcst_test = np.stack((dangjin_fcst_test,)*3, axis = -1)

ulsan_fcst_train = np.squeeze(ulsan_fcst_train, axis = -1)
ulsan_fcst_train = np.stack((ulsan_fcst_train,)*3, axis = -1)

ulsan_fcst_test = np.squeeze(ulsan_fcst_test, axis = -1)
ulsan_fcst_test = np.stack((ulsan_fcst_test,)*3, axis = -1)

In [26]:
dangjin_fcst_train.shape

(36458, 72, 16, 3)

* 의문 : image resize (72,32)가 왜 되는거지....? column 2개가 늘어났는데...?

In [27]:
'''
dangjin_fcst_train = tf.image.resize(dangjin_fcst_train, (72, 32)) #(36458, 72, 16, 1) -> TensorShape([36458, 72, 32, 1])
dangjin_fcst_test = tf.image.resize(dangjin_fcst_test, (72, 32))

ulsan_fcst_train = tf.image.resize(ulsan_fcst_train, (72, 32))
ulsan_fcst_test = tf.image.resize(ulsan_fcst_test, (72, 32))

dangjin_fcst_train = np.squeeze(dangjin_fcst_train.numpy(), axis = -1) #[36458, 72, 32, 1] => (36458, 72, 32)
dangjin_fcst_train = np.stack((dangjin_fcst_train,)*3, axis = -1)  #(36458, 72, 32, 3) => 3개 채널 똑같이 복사한듯 (Squeeze 한 이유가 이거인가 싶음)

dangjin_fcst_test = np.squeeze(dangjin_fcst_test.numpy(), axis = -1)
dangjin_fcst_test = np.stack((dangjin_fcst_test,)*3, axis = -1)

ulsan_fcst_train = np.squeeze(ulsan_fcst_train.numpy(), axis = -1)
ulsan_fcst_train = np.stack((ulsan_fcst_train,)*3, axis = -1)

ulsan_fcst_test = np.squeeze(ulsan_fcst_test.numpy(), axis = -1)
ulsan_fcst_test = np.stack((ulsan_fcst_test,)*3, axis = -1)
'''

'\ndangjin_fcst_train = tf.image.resize(dangjin_fcst_train, (72, 32)) #(36458, 72, 16, 1) -> TensorShape([36458, 72, 32, 1])\ndangjin_fcst_test = tf.image.resize(dangjin_fcst_test, (72, 32))\n\nulsan_fcst_train = tf.image.resize(ulsan_fcst_train, (72, 32))\nulsan_fcst_test = tf.image.resize(ulsan_fcst_test, (72, 32))\n\ndangjin_fcst_train = np.squeeze(dangjin_fcst_train.numpy(), axis = -1) #[36458, 72, 32, 1] => (36458, 72, 32)\ndangjin_fcst_train = np.stack((dangjin_fcst_train,)*3, axis = -1)  #(36458, 72, 32, 3) => 3개 채널 똑같이 복사한듯 (Squeeze 한 이유가 이거인가 싶음)\n\ndangjin_fcst_test = np.squeeze(dangjin_fcst_test.numpy(), axis = -1)\ndangjin_fcst_test = np.stack((dangjin_fcst_test,)*3, axis = -1)\n\nulsan_fcst_train = np.squeeze(ulsan_fcst_train.numpy(), axis = -1)\nulsan_fcst_train = np.stack((ulsan_fcst_train,)*3, axis = -1)\n\nulsan_fcst_test = np.squeeze(ulsan_fcst_test.numpy(), axis = -1)\nulsan_fcst_test = np.stack((ulsan_fcst_test,)*3, axis = -1)\n'

In [28]:
dangjin_fcst_train.shape

(36458, 72, 16, 3)

In [29]:
dangjin_fcst_train.shape[1:]

(72, 16, 3)

* EfficientNet 불러오기

In [30]:
def my_model():
  bottom = tf.keras.applications.EfficientNetB0(input_shape = dangjin_fcst_train.shape[1:], weights = None, include_top = False) #pre-trained weight 안가져옴, B0가 제일 가벼운 모델, 사진크기도 제일 작음
  model = tf.keras.models.Sequential([
                                      bottom,
                                      tf.keras.layers.GlobalAveragePooling2D(), #Q. 왜 GAP 먼저?? 그럼 한 데이터당 한 값인데.. => EfficientNet 뒤엔 보통 GAP.. 그리고 GAP 안하면 자꾸 에러남
                                      #tf.keras.layers.GlobalMaxPooling2D(),
                                      #tf.keras.layers.Dense(512, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.0001)),
                                      #tf.keras.layers.Dropout(0.3),
                                      #tf.keras.layers.Dense(256, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.0001)),
                                      #tf.keras.layers.Dropout(0.3),
                                      tf.keras.layers.Dense(1)  #마지막까지 relu로 하고(?) 하나로 숫자 뽑아서 이걸 regression처럼 쓴다고...;;
  ])
  return model

In [31]:
#Basic Conv2d => 성능 나쁘지 않았지만 EfficientNet 최고 성능이랑 비슷했음
def my_model(n_filters=10, fsize=5):
  new_model = tf.keras.Sequential()
  new_model.add(tf.keras.layers.Conv2D(n_filters, (1, fsize), padding='same', activation='relu', input_shape=dangjin_fcst_train.shape[1:]))
  new_model.add(tf.keras.layers.Flatten())
  new_model.add(tf.keras.layers.Dense(1000, activation='relu'))
  new_model.add(tf.keras.layers.Dropout(0.3))
  new_model.add(tf.keras.layers.Dense(500, activation='relu'))
  new_model.add(tf.keras.layers.Dropout(0.3))
  new_model.add(tf.keras.layers.Dense(100, activation='relu'))
  new_model.add(tf.keras.layers.Dense(1))
   
  return new_model

In [32]:
def make_dataset(df, col):
  X = df[energy[col].notnull() & energy[col]!=0]
  y = energy[energy[col].notnull() & energy[col]!=0]

  tmp = y.groupby(['year', 'month', 'day']).mean()
  tmp = tmp.query("%s == 0"%col).reset_index()
  tmp['year-month-day'] = tmp['year'].astype(str) + "-" + tmp['month'].astype(str) + "-" + tmp['day'].astype(str)

  X = X[~y['year-month-day'].isin(tmp['year-month-day'])]
  y = y[~y['year-month-day'].isin(tmp['year-month-day'])][col]
  return X, y.values

In [33]:
floating_X, floating_y = make_dataset(dangjin_fcst_train, "dangjin_floating")

In [36]:
#Fold 때문에 5번 돌아감
from keras import optimizers
BATCH_SIZE = 128
EPOCH = 70
kfold = KFold(n_splits = 5, shuffle = True, random_state = 42)
for i, (train_idx, val_idx) in enumerate(kfold.split(floating_X, floating_y)):

  X_train, X_val = floating_X[train_idx], floating_X[val_idx]
  y_train, y_val = floating_y[train_idx], floating_y[val_idx]
  
  model_path = '/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold%s.h5'%i
  model = my_model()
  #optim = tf.keras.optimizers.Adam(lr=0.0001)
  model.compile(optimizer = 'RMSProp', loss = "mse", metrics = ["mae"])
  model.fit(X_train, y_train, batch_size = BATCH_SIZE, epochs = EPOCH, validation_data = (X_val, y_val),
                      callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath = model_path, monitor = 'val_mae', save_best_only = True, mode = 'min'),
                                   tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_mae', factor = 0.5, patience = 5)])

Epoch 1/70
83/83 [==============================] - 3s 23ms/step - loss: 33264.7305 - mae: 144.0395 - val_loss: 34785.7305 - val_mae: 138.6039
Epoch 2/70
83/83 [==============================] - 2s 21ms/step - loss: 17669.1289 - mae: 98.2964 - val_loss: 10816.5918 - val_mae: 75.4786
Epoch 3/70
83/83 [==============================] - 2s 21ms/step - loss: 15882.2363 - mae: 92.6915 - val_loss: 10553.2715 - val_mae: 74.3525
Epoch 4/70
83/83 [==============================] - 2s 21ms/step - loss: 14125.2666 - mae: 86.2336 - val_loss: 9524.9551 - val_mae: 68.7194
Epoch 5/70
83/83 [==============================] - 2s 21ms/step - loss: 13218.2393 - mae: 81.8819 - val_loss: 9924.8535 - val_mae: 66.9389
Epoch 6/70
83/83 [==============================] - 2s 21ms/step - loss: 12674.2119 - mae: 80.1953 - val_loss: 8919.1201 - val_mae: 64.4380
Epoch 7/70
83/83 [==============================] - 2s 21ms/step - loss: 11912.4277 - mae: 77.3094 - val_loss: 9239.4785 - val_mae: 64.2783
Epoch 8/70
83/8

In [37]:
warehouse_X, warehouse_y = make_dataset(dangjin_fcst_train, "dangjin_warehouse")

In [38]:
# 당진 warehouse : 29.7에서 29.4 정도로 줄어듦.....; 

kfold = KFold(n_splits = 5, shuffle = True, random_state = 42)
for i, (train_idx, val_idx) in enumerate(kfold.split(warehouse_X, warehouse_y)):

  X_train, X_val = warehouse_X[train_idx], warehouse_X[val_idx]
  y_train, y_val = warehouse_y[train_idx], warehouse_y[val_idx]
  
  model_path = '/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold%s.h5'%i
  model = my_model()
  model.compile(optimizer = "adam", loss = "mse", metrics = ["mae"])
  model.fit(X_train, y_train, batch_size = BATCH_SIZE, epochs = EPOCH, validation_data = (X_val, y_val),
                      callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath = model_path, monitor = 'val_mae', save_best_only = True, mode = 'min'),
                                   tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_mae', factor = 0.5, patience = 7)])

Epoch 1/70
109/109 [==============================] - 3s 20ms/step - loss: 15455.9463 - mae: 96.2749 - val_loss: 6871.7461 - val_mae: 61.6012
Epoch 2/70
109/109 [==============================] - 2s 17ms/step - loss: 7038.2095 - mae: 61.2204 - val_loss: 6969.3115 - val_mae: 58.1388
Epoch 3/70
109/109 [==============================] - 2s 17ms/step - loss: 6781.2842 - mae: 59.5538 - val_loss: 6398.0283 - val_mae: 58.4427
Epoch 4/70
109/109 [==============================] - 2s 17ms/step - loss: 6429.5435 - mae: 57.4965 - val_loss: 6181.0708 - val_mae: 53.9132
Epoch 5/70
109/109 [==============================] - 2s 18ms/step - loss: 6352.1338 - mae: 56.9218 - val_loss: 5916.2319 - val_mae: 53.3247
Epoch 6/70
109/109 [==============================] - 2s 17ms/step - loss: 6095.8340 - mae: 55.4078 - val_loss: 5796.3486 - val_mae: 54.9096
Epoch 7/70
109/109 [==============================] - 2s 17ms/step - loss: 6042.4380 - mae: 55.2123 - val_loss: 5678.1533 - val_mae: 51.7332
Epoch 8/70
1

In [39]:
dangjin_X, dangjin_y = make_dataset(dangjin_fcst_train, "dangjin")

In [40]:

kfold = KFold(n_splits = 5, shuffle = True, random_state = 42)
for i, (train_idx, val_idx) in enumerate(kfold.split(dangjin_X, dangjin_y)):

  X_train, X_val = dangjin_X[train_idx], dangjin_X[val_idx]
  y_train, y_val = dangjin_y[train_idx], dangjin_y[val_idx]
  
  model_path = '/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold%s.h5'%i
  model = my_model()
  model.compile(optimizer = "adam", loss = "mse", metrics = ["mae"])
  model.fit(X_train, y_train, batch_size = BATCH_SIZE, epochs = EPOCH, validation_data = (X_val, y_val),
                      callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath = model_path, monitor = 'val_mae', save_best_only = True, mode = 'min'),
                                   tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_mae', factor = 0.5, patience = 7)])

Epoch 1/70
103/103 [==============================] - 3s 22ms/step - loss: 38116.3516 - mae: 154.6369 - val_loss: 17514.1992 - val_mae: 102.4757
Epoch 2/70
103/103 [==============================] - 2s 17ms/step - loss: 16460.2930 - mae: 96.6665 - val_loss: 15960.5352 - val_mae: 95.5796
Epoch 3/70
103/103 [==============================] - 2s 17ms/step - loss: 15344.4014 - mae: 92.2772 - val_loss: 15411.7148 - val_mae: 90.8534
Epoch 4/70
103/103 [==============================] - 2s 17ms/step - loss: 14656.0391 - mae: 89.8032 - val_loss: 14196.3018 - val_mae: 85.8042
Epoch 5/70
103/103 [==============================] - 2s 17ms/step - loss: 14325.4561 - mae: 88.3450 - val_loss: 16981.0352 - val_mae: 98.0967
Epoch 6/70
103/103 [==============================] - 2s 17ms/step - loss: 13608.1797 - mae: 85.3852 - val_loss: 13712.6113 - val_mae: 84.0779
Epoch 7/70
103/103 [==============================] - 2s 17ms/step - loss: 13426.4424 - mae: 84.7114 - val_loss: 14036.4902 - val_mae: 83.95

In [41]:
ulsan_X, ulsan_y = make_dataset(ulsan_fcst_train, "ulsan")

In [42]:
BATCH_SIZE = 128
kfold = KFold(n_splits = 5, shuffle = True, random_state = 42)
for i, (train_idx, val_idx) in enumerate(kfold.split(ulsan_X, ulsan_y)):

  X_train, X_val = ulsan_X[train_idx], ulsan_X[val_idx]
  y_train, y_val = ulsan_y[train_idx], ulsan_y[val_idx]
  
  model_path = '/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold%s.h5'%i
  model = my_model()
  model.compile(optimizer = "adam", loss = "mse", metrics = ["mae"])
  model.fit(X_train, y_train, batch_size = BATCH_SIZE, epochs = EPOCH, validation_data = (X_val, y_val),
                      callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath = model_path, monitor = 'val_mae', save_best_only = True, mode = 'min'),
                                   tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_mae', factor = 0.5, patience = 7)])

Epoch 1/70
107/107 [==============================] - 3s 21ms/step - loss: 6678.1865 - mae: 62.5226 - val_loss: 3320.1887 - val_mae: 42.3105
Epoch 2/70
107/107 [==============================] - 2s 17ms/step - loss: 2838.2026 - mae: 39.0984 - val_loss: 3046.5237 - val_mae: 40.9421
Epoch 3/70
107/107 [==============================] - 2s 17ms/step - loss: 2677.3730 - mae: 37.3277 - val_loss: 2520.9458 - val_mae: 35.3687
Epoch 4/70
107/107 [==============================] - 2s 19ms/step - loss: 2555.4187 - mae: 36.1558 - val_loss: 2537.1091 - val_mae: 35.5759
Epoch 5/70
107/107 [==============================] - 2s 18ms/step - loss: 2525.4617 - mae: 35.6591 - val_loss: 2605.4099 - val_mae: 37.0405
Epoch 6/70
107/107 [==============================] - 2s 17ms/step - loss: 2558.2490 - mae: 35.9190 - val_loss: 2363.4243 - val_mae: 34.6088
Epoch 7/70
107/107 [==============================] - 2s 17ms/step - loss: 2402.6597 - mae: 34.6280 - val_loss: 6195.1694 - val_mae: 62.0981
Epoch 8/70
10

In [43]:
floating_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold0.h5")
floating_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold1.h5")
floating_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold2.h5")
floating_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold3.h5")
floating_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold4.h5")

In [44]:
floating0 = floating_model0.predict(dangjin_fcst_test)
floating1 = floating_model1.predict(dangjin_fcst_test)
floating2 = floating_model2.predict(dangjin_fcst_test)
floating3 = floating_model3.predict(dangjin_fcst_test)
floating4 = floating_model4.predict(dangjin_fcst_test)

In [45]:
warehouse_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold0.h5")
warehouse_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold1.h5")
warehouse_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold2.h5")
warehouse_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold3.h5")
warehouse_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold4.h5")

In [46]:
warehouse0 = warehouse_model0.predict(dangjin_fcst_test)
warehouse1 = warehouse_model1.predict(dangjin_fcst_test)
warehouse2 = warehouse_model2.predict(dangjin_fcst_test)
warehouse3 = warehouse_model3.predict(dangjin_fcst_test)
warehouse4 = warehouse_model4.predict(dangjin_fcst_test)

In [47]:
dangjin_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold0.h5")
dangjin_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold1.h5")
dangjin_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold2.h5")
dangjin_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold3.h5")
dangjin_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold4.h5")

In [48]:
dangjin0 = dangjin_model0.predict(dangjin_fcst_test)
dangjin1 = dangjin_model1.predict(dangjin_fcst_test)
dangjin2 = dangjin_model2.predict(dangjin_fcst_test)
dangjin3 = dangjin_model3.predict(dangjin_fcst_test)
dangjin4 = dangjin_model4.predict(dangjin_fcst_test)

In [49]:
ulsan_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold0.h5")
ulsan_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold1.h5")
ulsan_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold2.h5")
ulsan_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold3.h5")
ulsan_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold4.h5")

In [50]:
ulsan0 = ulsan_model0.predict(ulsan_fcst_test)
ulsan1 = ulsan_model1.predict(ulsan_fcst_test)
ulsan2 = ulsan_model2.predict(ulsan_fcst_test)
ulsan3 = ulsan_model3.predict(ulsan_fcst_test)
ulsan4 = ulsan_model4.predict(ulsan_fcst_test)

In [51]:
floating = (floating0 + floating1 + floating2 + floating3 + floating4) / 5
warehouse = (warehouse0 + warehouse1 + warehouse2 + warehouse3 + warehouse4) / 5
dangjin = (dangjin0 + dangjin1 + dangjin2 + dangjin3 + dangjin4) / 5
ulsan = (ulsan0 + ulsan1 + ulsan2 + ulsan3 + ulsan4) / 5

In [52]:
sample_submission = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/sample_submission.csv")

In [53]:
sample_submission.loc[:671, 'dangjin_floating'] = floating
sample_submission.loc[:671, 'dangjin_warehouse'] = warehouse
sample_submission.loc[:671, 'dangjin'] = dangjin
sample_submission.loc[:671, 'ulsan'] = ulsan

findfont: Font family ['NanumBarunGothic'] not found. Falling back to DejaVu Sans.


In [58]:
# notzero
plt.figure(figsize = (30, 10))
plt.plot(sample_submission['dangjin_floating'][:672])
plt.plot(sample_submission['dangjin_warehouse'][:672])
plt.plot(sample_submission['dangjin'][:672])
plt.plot(sample_submission['ulsan'][:672])

In [54]:
sample_submission.to_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/0608_cnn_5fold.csv", index = False)

In [ ]:
ls

0608_cnn_5fold.csv  gdrive/  sample_data/
